In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 

In [90]:
df=pd.read_csv("Speed_Dating_Data.csv", encoding = "ISO-8859-1")

In [91]:
list_qual=["attr","sinc","intel","fun","amb","shar","dec",'iid','id','gender','age','field','field_cd','race','imprace','imprelig','date','go_out',
   'sports','tvsports','exercise','dining','museums','art','hiking','gaming','clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga',
   'attr1_1', 'sinc1_1', 'intel1_1','fun1_1', 'amb1_1', 'shar1_1','attr3_1', 'sinc3_1', 'intel3_1','fun3_1', 'amb3_1',"pid"]
list_var=[i for i in df.columns if ((any(x in i for x in list_qual)&("1_1" in i))| (i in list_qual))]
df2=df[(df.wave<6)|(df.wave>9)]
df2=df[list_qual]
df2["somme"]=df2.attr1_1+ df2.sinc1_1+df2.intel1_1+df2.amb1_1+df2.shar1_1 +df2.fun1_1
df2=df2[(df2.somme<110)&(df2.somme>95)].drop("somme",1)
df2=df2.dropna()
df2.pid=df2.pid.astype(int)


C:\Users\maria\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [115]:
list_var=['age','field','field_cd','race','imprace','imprelig','date','go_out','sports','tvsports','exercise',
          'dining','museums','art','hiking','gaming','clubbing','reading','tv','theater','movies','concerts','music',
          'shopping','yoga','attr3_1', 'sinc3_1', 'intel3_1','fun3_1', 'amb3_1',"iid"]

df3=df2[list_var]
dic={i:i+"_partner" for i in list_var[:-1]}
df3 = df3.rename(columns=dic)
df3=df3.rename(columns={"iid":"pid"})
df3=df3.drop_duplicates()

In [124]:
df4=pd.merge(df3,df2,how="outer").dropna()

In [126]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.tree import export_graphviz

In [127]:
def split(frac,df):
    train=df.sample(frac=frac,random_state=200)
    test=df.drop(train.index)
    return(train,test)
def XY(frac,df):
    train,test=split(frac,df)
    Ytrain=train.dec.values
    Ytest=test.dec.values
    Xtrain=train.drop("dec",1).values
    Xtest=test.drop("dec",1).values
    return(Xtrain,Xtest,Ytrain,Ytest)
def n_estimators(n,frac,df):
    clf=RandomForestClassifier(n_estimators=n)
    train,test=split(frac,df)
    clf.fit(XY(train,test)[0], XY(train,test)[2])
    Ypred=clf.predict(XY(train,test)[1])
    return(f1_score(XY(train,test)[3],Ypred ))

In [128]:
Xtrain,Xtest,Ytrain,Ytest=XY(0.8,df4)

In [129]:
l=[]
m=[]
for i in range (20):
    Xtrain,Xtest,Ytrain,Ytest=XY(0.8,df4)
    clf=RandomForestClassifier()
    clf.fit(Xtrain,Ytrain)
    Ypred=clf.predict(Xtest)
    l.append(f1_score(Ytest,Ypred))
    m.append(clf.score(Xtrain,Ytrain))

ValueError: could not convert string to float: 'Public Administration'

In [ ]:
print(np.mean(l),np.mean(m))